# Metodo de Newton para solucionar el Solver de Markovitz

In [0]:
import numpy as np
import cupy as cp
import solver_2 as mkv

## Definimos funciones

In [0]:
def inc_index(vec,index,h):
    '''
    Auxiliary function for gradient and Hessian computation.
    Args:
        vec (array): numpy array.
        index (int): index.
        h (float):   quantity that vec[index] will be increased.
    Returns:
        vec (array): numpy array vec with vec[index] increased by h.
    '''
    vec[index] +=h
    return vec

In [0]:
def dec_index(vec,index,h=1):
    '''
    Auxiliary function for gradient and Hessian computation.
    Args:
        vec (array): numpy array.
        index (int): index.
        h (float):   quantity that vec[index] will be decreased.
    Returns:
        vec (array): numpy array vec with vec[index] decreased by h.
    '''
    vec[index] -=h
    return vec

In [0]:
def compute_error(x_obj,x_approx):
    '''
    Relative or absolute error between x_obj and x_approx.
    '''
    if np.linalg.norm(x_obj) > np.nextafter(0,1):
        Err=np.linalg.norm(x_obj-x_approx)/np.linalg.norm(x_obj)
    else:
        Err=np.linalg.norm(x_obj-x_approx)
    return Err

In [0]:
def norm_residual(feas_primal, feas_dual):
    '''
    Computes norm of residual for Newtons infeasible initial point method
    '''
    return np.sqrt(np.linalg.norm(feas_primal)**2 +\
                   np.linalg.norm(feas_dual)**2
                  )

In [0]:
def gradient_approximation(f,x,h=1e-8):
    '''
    Numerical approximation of gradient for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where gradient will be computed.
        h (float): step size for forward differences, tipically h=1e-8
    Returns:
        gf (array): numerical approximation to gradient of f.
    '''
    n = x.size
    gf = np.zeros(n)
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        gf[i] = f(x) - f_x
        dec_index(x,i,h)
    return gf/h

In [0]:
def Hessian_approximation(f,x,h=1e-6):
    '''
    Numerical approximation of Hessian for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where Hessian will be computed.
        h (float): step size for forward differences, tipically h=1e-6
    Returns:
        Hf (array): numerical approximation to Hessian of f.
    '''
    n = x.size
    Hf = np.zeros((n,n))
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        f_x_inc_in_i = f(x)
        for j in np.arange(i,n):
            inc_index(x,j,h)
            f_x_inc_in_i_j = f(x)
            dec_index(x,i,h)
            f_x_inc_in_j = f(x)
            dif = f_x_inc_in_i_j-f_x_inc_in_i-f_x_inc_in_j+f_x
            Hf[i,j] = dif
            if j != i:
                Hf[j,i] = dif
            dec_index(x,j,h)
            inc_index(x,i,h)
        dec_index(x,i,h)
    return Hf/h**2

In [0]:
def line_search_by_backtracking(f,dir_desc,x,
                                der_direct, alpha=.15, beta=.5):
    '''
    Line search that sufficiently decreases f restricted to a ray in the direction dir_desc.
    Args:
        alpha (float): parameter in line search with backtracking, tipically .15
        beta (float): parameter in line search with backtracking, tipically .5
        f (lambda expression): definition of function f.
        dir_desc (array): descent direction.
        x (array): numpy array that holds values where line search will be performed.
        der_direct (float): directional derivative of f.
    Returns:
        t (float): positive number for stepsize along dir_desc that sufficiently decreases f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha must be less than or equal to 1/2')
        t=-1
    if beta>1:
        print('beta must be less than 1')
        t=-1;   
    if t!=-1:
        eval1 = f(x+t*dir_desc)
        eval2 = f(x) + alpha*t*der_direct
        while eval1 > eval2:
            t=beta*t
            eval1=f(x+t*dir_desc)
            eval2=f(x)+alpha*t*der_direct
    else:
        t=-1
    return t

In [0]:
def line_search_for_residual_by_backtracking(r_primal, r_dual,dir_desc_primal,dir_desc_dual,x, nu,
                                             norm_residual_eval,
                                             alpha=.15, beta=.5):
    '''
    Line search that sufficiently decreases residual for Newtons infeasible initial point method
    restricted to a ray in the direction dir_desc.
    Args:
        r_primal (fun): definition of primal residual as function definition or lambda expression.
        r_dual (fun): definition of dual residual as function definition or lambda expression.
        dir_desc_primal (array): descent direction for primal variable.
        dir_desc_dual (array): descent direction for dual variable.
        x (array): numpy array that holds values where line search will be performed.
        nu (array): numpy array that holds values where line search will be performed.
        norm_residual_eval (float): norm of residual that has both r_primal and r_dual evaluations in
                                    x and nu
        alpha (float): parameter in line search with backtracking, tipically .15
        beta (float): parameter in line search with backtracking, tipically .5
        
    Returns:
        t (float): positive number for stepsize along dir_desc that sufficiently decreases f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha must be less than or equal to 1/2')
        t=-1
    if beta>1:
        print('beta must be less than 1')
        t=-1;   
    if t!=-1:
        feas_primal = r_primal(x + t*dir_desc_primal)
        feas_dual = r_dual(nu + t*dir_desc_dual )
        eval1 = norm_residual(feas_primal, feas_dual)
        eval2 = (1-alpha*t)*norm_residual_eval
        while eval1 > eval2:
            t=beta*t
            feas_primal = r_primal(x + t*dir_desc_primal)
            feas_dual = r_dual(nu + t*dir_desc_dual )
            eval1 = norm_residual(feas_primal, feas_dual)
            eval2 = (1-alpha*t)*norm_residual_eval
    return t

In [0]:
def Newtons_method_feasible_init_point(f, A, x_0, tol, 
                                       tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                       gf_symbolic = None,
                                       Hf_symbolic = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    IMPORTANT: this implementation requires that initial point x_0, satisfies: Ax_0 = b
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        x_0 (numpy ndarray): initial point for Newton's method. Must satisfy: Ax_0 = b
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via finite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
        
    x = x_0
    
    feval = f(x)
    
    if gf_symbolic:
        gfeval = gf_symbolic(x)
    else:
        gfeval = gradient_approximation(f,x)

    if Hf_symbolic:
        Hfeval = Hf_symbolic(x)
    else:
        Hfeval = Hessian_approximation(f,x)
    
    normgf = np.linalg.norm(gfeval)
    condHf= np.linalg.cond(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]=compute_error(p_ast,feval)
    
    Err = compute_error(x_ast,x)
    
        
    if(A.ndim == 1):
        p = 1
        n = x.size
        zero_matrix = np.zeros(p)
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
    else:
        p,n = A.shape
        zero_matrix = np.zeros((p,p))
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.column_stack((A,zero_matrix))
        
    x_plot = np.zeros((n,maxiter))
    x_plot[:,iteration] = x
    
    system_matrix = np.row_stack((first_stack,second_stack))
    zero_vector = np.zeros(p)
    rhs = np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]

    #Newton's direction and Newton's decrement
    dir_desc = np.linalg.solve(system_matrix, -rhs)
    dir_Newton = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton)
    w_dual_variable_estimation = dir_desc[n:(n+p)]


    print('I\tNormgf \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{}\t\t{:0.2e}'.format(iteration,normgf,
                                                                         dec_Newton,Err,
                                                                         Err_plot_aux[iteration],"---",
                                                                         condHf))
    
    stopping_criteria = dec_Newton/2
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = line_search_by_backtracking(f,dir_Newton,x,der_direct)
        x = x + t*dir_Newton
        feval = f(x)
        
        
        if gf_symbolic:
            gfeval = gf_symbolic(x)
        else:
            gfeval = gradient_approximation(f,x)
        
        if Hf_symbolic:
            Hfeval = Hf_symbolic(x)
        else:
            Hfeval = Hessian_approximation(f,x)
        if(A.ndim == 1):
            first_stack = np.column_stack((Hfeval, A.T))
        else:
            first_stack = np.column_stack((Hfeval, A.T))

        system_matrix = np.row_stack((first_stack,second_stack))
        rhs = np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]
        #Newton's direction and Newton's decrement
        dir_desc = np.linalg.solve(system_matrix, -rhs)
        dir_Newton = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton)
        w_dual_variable_estimation = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]=compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = compute_error(x_ast,x)
        print('{}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}'.format(iteration,normgf,
                                                                                dec_Newton,Err,
                                                                                Err_plot_aux[iteration],t,
                                                                                condHf))
        stopping_criteria = dec_Newton/2
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {:0.2e}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
    else:
        if iteration == maxiter:
            print("Reached maximum of iterations, check approximation")
    x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]

## Cargamos la información y calculamos los parámetros

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [14]:
datos = mkv.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [0]:
mu = mkv.calcular_rendimiento(datos)

In [0]:
S = mkv.calcular_varianza(datos)

In [102]:
r=max(mu)
r

array(0.40221088)

## Resolvemos con el Método de Newton

In [0]:
fo = lambda w: w@S@w

In [104]:
w_ast = mkv.markowitz(r,mu,S)
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

In [105]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618598,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806557,  0.26292508,
         0.07357148,  0.06803817,  0.03867589,  0.05090149,  0.14401416,
        -0.09339291,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274014, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.0717407 , -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

In [106]:
b = cp.array([r.item(),1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial como:

In [107]:
w_0 = cp.zeros(n)
w_0[0] = 1
w_0

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [0]:
## Este bloque es temporal
A = cp.asnumpy(A)
w_0 = cp.asnumpy(w_0)
w_ast = cp.asnumpy(w_ast)
p_ast = cp.asnumpy(p_ast)
mu = cp.asnumpy(mu)
S = cp.asnumpy(S)
fo = lambda w: w@S@w

In [110]:
[x,total_of_iterations,Err_plot,x_plot]=Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	6.58e-04	4.17e-04	1.05e+00	1.69e+00	---		9.52e+03
1	6.58e-04	4.37e-10	4.92e-01	5.15e-01	1.00e+00	9.52e+03
Error of x with respect to x_ast: 4.92e-01
Approximate solution: [ 0.11020792 -0.00181225  0.03226334  0.03682051  0.06270992  0.08126923
  0.02139943  0.07009044 -0.02162838  0.10847652  0.00476652 -0.00236751
  0.17026441  0.08068742 -0.02897799  0.00344072  0.00445215  0.03330976
  0.03845186  0.07399891  0.02339894  0.07876919 -0.0278804   0.07535552
  0.04259247  0.05794904  0.02488739 -0.01496674  0.02098573 -0.02318579
  0.17491674 -0.05645838  0.05508702  0.06523369 -0.04426473 -0.0440297
  0.09055173  0.03463329  0.02737469 -0.0349971  -0.03933259  0.02130088
  0.04398815  0.01553946  0.00244568 -1.29941471  0.56057215  0.18295137
  0.09398698  0.01418713]


In [111]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

In [114]:
x@S@x

4.5884983038880295e-05

In [115]:
w_ast@S@w_ast

9.459792459072343e-05

In [116]:
x@mu

0.13254293166879796

In [117]:
w_ast@mu

0.4022108787760793

In [118]:
sum(x)

1.0